In [139]:
import os
import glob
import collections
import pandas as pd
from mlxtend.frequent_patterns import apriori as mx_apriori
from mlxtend.frequent_patterns import association_rules as mx_association_rules

path = "./projects/result"
filenames = glob.glob(path + "/*.csv")

item_list = []

for filename in filenames:
    df = pd.read_csv(filename)
    item_list.append(df[df['detection_scores'] >= 0.2]['detection_class_entities'].apply(lambda x: x[2:-1]).tolist())

result = sum(item_list, [])


counter=collections.Counter(result)
delete_list = ['Clothing','Human face','Person','Human head','Human hand','Human hair','Human eye','Human nose','Human beard',]

animal_item_list = []
for list in item_list:
    if 'Horse' in list:
        for item in delete_list:
            while (item in list):
                try:
                    list.remove(item)
                except:
                    pass
        animal_item_list.append(list)

print(len(animal_item_list))

44


## Support

In [140]:
def print_support_mx(df,max_display=10,min_items=1):
    print('support\t itemset')
    print('-'*30)
    lenrow = df['itemsets'].apply(lambda x: len(x))
    df  = df[lenrow>=min_items]
    df  = df.sort_values('support',ascending=False).iloc[:max_display]
    for i,row in df.iterrows():
        print("%.2f" % float(row['support']),'\t',set(row['itemsets']))

In [141]:
df_dummy = pd.get_dummies(pd.Series(animal_item_list).apply(pd.Series).stack()).sum(level=0)
df_dummy = df_dummy.apply(lambda x: x.replace(2, 1))
df_dummy = df_dummy.apply(lambda x: x.replace(3, 1))
df_dummy = df_dummy.apply(lambda x: x.replace(4, 1))
df_dummy = df_dummy.apply(lambda x: x.replace(5, 1))
df_dummy = df_dummy.apply(lambda x: x.replace(6, 1))
df_dummy = df_dummy.apply(lambda x: x.replace(7, 1))
df_dummy = df_dummy.apply(lambda x: x.replace(8, 1))
df_dummy = df_dummy.apply(lambda x: x.replace(9, 1))
df_dummy = df_dummy.apply(lambda x: x.replace(10, 1))
df_dummy = df_dummy.apply(lambda x: x.replace(11, 1))
df_dummy = df_dummy.apply(lambda x: x.replace(12, 1))
frequent_itemsets = mx_apriori(df_dummy, min_support=0.01, use_colnames=True)
print_support_mx(frequent_itemsets,100,min_items=2)

support	 itemset
------------------------------
0.41 	 {'Horse', 'Woman'}
0.36 	 {'Man', 'Horse'}
0.32 	 {'Footwear', 'Horse'}
0.25 	 {'Horse', 'Tree'}
0.23 	 {'Animal', 'Horse'}
0.16 	 {'Man', 'Footwear'}
0.16 	 {'Man', 'Horse', 'Footwear'}
0.16 	 {'Horse', 'Dog'}
0.14 	 {'Man', 'Horse', 'Woman'}
0.14 	 {'Footwear', 'Tree'}
0.14 	 {'Man', 'Woman'}
0.14 	 {'Footwear', 'Horse', 'Tree'}
0.09 	 {'Animal', 'Horse', 'Woman'}
0.09 	 {'Tree', 'Woman'}
0.09 	 {'Footwear', 'Woman'}
0.09 	 {'Horse', 'Tree', 'Woman'}
0.09 	 {'Dog', 'Woman'}
0.09 	 {'Animal', 'Man'}
0.09 	 {'Horse', 'Dog', 'Woman'}
0.09 	 {'Man', 'Tree'}
0.09 	 {'Animal', 'Woman'}
0.09 	 {'Footwear', 'Horse', 'Woman'}
0.09 	 {'Man', 'Horse', 'Tree'}
0.09 	 {'Animal', 'Man', 'Horse'}
0.07 	 {'Man', 'Dog'}
0.07 	 {'Animal', 'Horse', 'Sculpture'}
0.07 	 {'Horse', 'Fashion accessory', 'Woman'}
0.07 	 {'Animal', 'Dog'}
0.07 	 {'Animal', 'Sculpture'}
0.07 	 {'Man', 'Horse', 'Dog'}
0.07 	 {'Fashion accessory', 'Woman'}
0.07 	 {'Horse', '

/var/folders/f8/0gc63p0s55g3jx1xwrb_vh7w0000gn/T/ipykernel_8113/2531296464.py:1: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
  df_dummy = pd.get_dummies(pd.Series(animal_item_list).apply(pd.Series).stack()).sum(level=0)


## Confidence

In [142]:
def print_rules_mx(df,max_display=10):
    print('confidence\t rule')
    print('-'*30)
    df  = df.sort_values('confidence',ascending=False).iloc[:max_display]
    for i,row in df.iterrows():
        print("%.2f" % float(row['confidence']),'\t',set(row['antecedents']),'->',set(row['consequents']))

In [143]:
rules_mx = mx_association_rules(frequent_itemsets, metric="confidence", min_threshold=0.1)
print_rules_mx(rules_mx,max_display=100)

confidence	 rule
------------------------------
1.00 	 {'Animal', 'Umbrella'} -> {'Man', 'Horse', 'Footwear', 'Woman'}
1.00 	 {'Bird', 'Dog', 'Woman'} -> {'Animal', 'Cattle'}
1.00 	 {'Animal', 'Bird', 'Dog', 'Woman'} -> {'Cattle'}
1.00 	 {'Animal', 'Cattle', 'Dog', 'Woman'} -> {'Bird'}
1.00 	 {'Cattle', 'Bird', 'Dog', 'Woman'} -> {'Animal'}
1.00 	 {'Animal', 'Bird', 'Cattle', 'Dog'} -> {'Woman'}
1.00 	 {'Animal', 'Bird', 'Woman'} -> {'Cattle', 'Dog'}
1.00 	 {'Animal', 'Cattle', 'Woman'} -> {'Bird', 'Dog'}
1.00 	 {'Cattle', 'Bird', 'Woman'} -> {'Animal', 'Dog'}
1.00 	 {'Cattle', 'Dog', 'Woman'} -> {'Animal', 'Bird'}
1.00 	 {'Animal', 'Man', 'Footwear', 'Woman'} -> {'Horse', 'Umbrella'}
1.00 	 {'Animal', 'Bird', 'Cattle'} -> {'Dog', 'Woman'}
1.00 	 {'Animal', 'Bird', 'Dog'} -> {'Cattle', 'Woman'}
1.00 	 {'Animal', 'Cattle', 'Dog'} -> {'Bird', 'Woman'}
1.00 	 {'Cattle', 'Bird', 'Dog'} -> {'Animal', 'Woman'}
1.00 	 {'Bird', 'Woman'} -> {'Animal', 'Cattle', 'Dog'}
1.00 	 {'Cattle', 'Woman'}